# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from config import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/clean_city_data.csv")
cities.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidty,Cloudiness,Wind Speed,Country,Date
0,0,eenhana,-17.47,16.33,51.80,40,0,1.12,NaN,1594684006
1,1,coahuayana,18.73,-103.68,82.40,69,40,11.41,MX,1594683784
2,2,tuktoyaktuk,69.45,-133.04,62.60,63,40,6.93,CA,1594684007
3,3,port elizabeth,-33.92,25.57,53.51,51,50,20.33,ZA,1594684007
4,4,cutervo,-6.38,-78.82,50.74,100,51,2.80,PE,1594684007


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities["Humidty"]
locations = cities[["Lat","Lng"]]
maxhumidity = humidity.max()

In [4]:
#Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = maxhumidity, point_radius = 5)

#Add Layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80)& (cities["Wind Speed"] < 10), :]
city_df = city_df.dropna(how='any')
city_df.reset_index(inplace=True)
del city_df['index']
city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidty,Cloudiness,Wind Speed,Country,Date
0,5,port hawkesbury,45.62,-61.35,78.01,78,75,4.70,CA,1594684007
1,7,zhangjiakou,40.81,114.88,70.23,64,62,2.91,CN,1594684008
2,15,biak,-0.91,122.88,78.67,81,92,4.65,ID,1594684009
3,27,gimli,50.63,-96.99,73.00,99,84,5.46,CA,1594683889
4,30,banda aceh,5.56,95.32,78.48,81,96,3.71,ID,1594684012
...,...,...,...,...,...,...,...,...,...,...
119,564,maneadero,31.72,-116.57,75.00,79,26,3.00,MX,1594684217
120,566,ariquemes,-9.91,-63.04,78.93,69,67,1.74,BR,1594684217
121,567,tutoia,-2.76,-42.27,77.68,82,100,8.88,BR,1594684218
122,569,montpellier,43.61,3.88,77.00,44,35,6.93,FR,1594684218


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_list= []
#Store into variable named hotel_df
hotel_df = city_df.copy()
for x in range(len(narrowed_city_df)):
    lat = city_df.loc[x]['Lat']
    lng = city_df.loc[x]['Lng']
    
    params = {
        "location":f"{lat},{lng}",
        "radius" : 5000,
        "types" : "hotel",
        "key" : g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    hotel= response.json()
#print(json.dumps(hotel, indent=4, sort_keys=True))
    try:
        hotel_list.append(hotel[
            "results"][0]["name"])
    except:
        hotel_list.append("")

# Adding a "Hotel Name" column to the Data Frame
city_df["Hotel Name"] = hotel_df
city_df = city_df.dropna(how = "any")


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidty,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,5,port hawkesbury,45.62,-61.35,78.01,78,75,4.70,CA,1594684007,5
1,7,zhangjiakou,40.81,114.88,70.23,64,62,2.91,CN,1594684008,7
2,15,biak,-0.91,122.88,78.67,81,92,4.65,ID,1594684009,15
3,27,gimli,50.63,-96.99,73.00,99,84,5.46,CA,1594683889,27
4,30,banda aceh,5.56,95.32,78.48,81,96,3.71,ID,1594684012,30


In [24]:
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidty,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,5,port hawkesbury,45.62,-61.35,78.01,78,75,4.70,CA,1594684007,5
1,7,zhangjiakou,40.81,114.88,70.23,64,62,2.91,CN,1594684008,7
2,15,biak,-0.91,122.88,78.67,81,92,4.65,ID,1594684009,15
3,27,gimli,50.63,-96.99,73.00,99,84,5.46,CA,1594683889,27
4,30,banda aceh,5.56,95.32,78.48,81,96,3.71,ID,1594684012,30


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))